In [ ]:
import os

from tqdm import tqdm

import jax
import jax.numpy as jnp
from jax import random, vmap

from jax.experimental import mesh_utils, multihost_utils
from jax.sharding import Mesh, PartitionSpec as P

from function_diffusion.models import Encoder, Decoder, DiT

from function_diffusion.utils.model_utils import (
    create_autoencoder_state,
    create_diffusion_state,
    create_optimizer,
    compute_total_params,
)
from function_diffusion.utils.train_utils import  sample_ode
from function_diffusion.utils.data_utils import create_dataloader
from function_diffusion.utils.checkpoint_utils import (
    create_checkpoint_manager,
    restore_checkpoint,
)

from model_utils import create_encoder_step, create_decoder_step
from data_utils import create_dataset

In [ ]:
from configs import diffusion

config = diffusion.get_config('fae,dit')

In [ ]:
def restore_fae_state(config, encoder, decoder):
    # Create learning rate schedule and optimizer
    lr, tx = create_optimizer(config)

    # Create train state
    state = create_autoencoder_state(config, encoder, decoder, tx)

    # Create checkpoint manager
    fae_job_name = f"{config.autoencoder.model_name}_use_pde_{config.training.use_pde}"

    ckpt_path = os.path.join(os.getcwd(), fae_job_name, "ckpt")
    ckpt_mngr = create_checkpoint_manager(config.saving, ckpt_path)

    # Restore the model from the checkpoint
    fae_state = restore_checkpoint(ckpt_mngr, state)
    print(f"Restored model {fae_job_name} from step", fae_state.step)

    return fae_state


In [ ]:
# Initialize function autoencoder
encoder = Encoder(**config.autoencoder.encoder)
decoder = Decoder(**config.autoencoder.decoder)

fae_state = restore_fae_state(config, encoder, decoder)

In [ ]:
# Initialize diffusion model
dit = DiT(**config.diffusion)
# Create learning rate schedule and optimizer
lr, tx = create_optimizer(config)

# Create diffusion train state
state = create_diffusion_state(config, dit, tx, use_conditioning=True)
num_params = compute_total_params(state)
print(f"Model storage cost: {num_params * 4 / 1024 / 1024:.2f} MB of parameters")

In [ ]:
# Create checkpoint manager
job_name = f"{config.diffusion.model_name}_use_pde_{config.training.use_pde}"
ckpt_path = os.path.join(os.getcwd(), job_name, "ckpt")
# Create checkpoint manager
ckpt_mngr = create_checkpoint_manager(config.saving, ckpt_path)

# Restore the model from the checkpoint
state = restore_checkpoint(ckpt_mngr, state)
print(f"Restored model {job_name} from step", state.step)

In [ ]:
# Device count
num_local_devices = jax.local_device_count()
num_devices = jax.device_count()
print(f"Number of devices: {num_devices}")
print(f"Number of local devices: {num_local_devices}")

# Create sharding for data parallelism
mesh = Mesh(mesh_utils.create_device_mesh((jax.device_count(),)), "batch")
state = multihost_utils.host_local_array_to_global_array(state, mesh, P())
fae_state = multihost_utils.host_local_array_to_global_array(fae_state, mesh, P())

In [ ]:
# Create encoder and decoder steps
encoder_step = create_encoder_step(encoder, mesh)
decoder_step = create_decoder_step(decoder, mesh)

In [ ]:
# Get test dataset
_, test_dataset = create_dataset(config)
test_loader = create_dataloader(test_dataset,
                                batch_size=4,
                                num_workers=config.dataset.num_workers,
                                shuffle=False)

In [ ]:
# Create uniform grid for evaluation
h, w = 200, 200

x_coords = jnp.linspace(0, 1, h)
y_coords = jnp.linspace(0, 1, w)
x_coords, y_coords = jnp.meshgrid(x_coords, y_coords, indexing='ij')
coords = jnp.hstack([x_coords.reshape(-1, 1), y_coords.reshape(-1, 1)])
coords = multihost_utils.host_local_array_to_global_array(coords, mesh, P())

In [ ]:
rng_key = jax.random.PRNGKey(888)

d = 5   # downsampling factor  # [1, 2, 5]
noise_level = 0.2

u_pred_list = []
u_true_list = []
r_pred_list = []
u_downsampled_list = []

iters = 0 
for batch in tqdm(test_loader):
    iters += 1

    rng_key, *keys = random.split(rng_key, 3)
    
    batch = jax.tree.map(jnp.array, batch)

    u = batch
    u_downsammpled = batch[:, ::d, ::d]

    noise = random.normal(keys[0], u_downsammpled.shape) * 0.2 * noise_level
    u_downsammpled = u_downsammpled + noise

    u_batch = (jnp.ones_like(u), u, jnp.ones_like(u))
    c_batch = (jnp.ones_like(u_downsammpled), u_downsammpled, jnp.ones_like(u_downsammpled))

    print(u_batch[0].shape, u_batch[1].shape, u_batch[2].shape)
    print(c_batch[0].shape, c_batch[1].shape, c_batch[2].shape)

    # # Shard the batch across devices
    u_batch = multihost_utils.host_local_array_to_global_array(
        u_batch, mesh, P("batch")
        )
    c_batch = multihost_utils.host_local_array_to_global_array(
        c_batch, mesh, P("batch")
        )

    z_u = encoder_step(fae_state.params[0], u_batch)
    z_c = encoder_step(fae_state.params[0], c_batch)

    z0 = random.normal(keys[1], shape=z_u.shape)

    z1_new, _ = sample_ode(state, z0=z0, c=z_c, num_steps=100, use_conditioning=True)  
    u_pred, r_pred = decoder_step(fae_state.params[1], z1_new, coords)

    u_pred = u_pred.reshape(-1, h, w)
    u_true = u.reshape(-1, h, w)
    r_pred = r_pred.reshape(-1, h, w)

    u = u.reshape(-1, h, w)
    u_downsammpled = u_downsammpled.reshape(-1, h//d, w//d)

    u_pred_list.append(u_pred)
    u_true_list.append(u)
    r_pred_list.append(r_pred)
    u_downsampled_list.append(u_downsammpled)

    if iters ==4:
        break

u_pred = jnp.vstack(u_pred_list)
u_true = jnp.vstack(u_true_list)
r_pred = jnp.vstack(r_pred_list)
u_downsammpled = jnp.vstack(u_downsampled_list)

In [ ]:
def compute_error(pred, y):
    return jnp.linalg.norm(pred.flatten() - y.flatten()) / jnp.linalg.norm(y.flatten())

error = vmap(compute_error)(u_pred, u_true)

print("Mean relative error:", jnp.mean(error))
print("Max relative error:", jnp.max(error))
print("Min relative error:", jnp.min(error))
print("Std relative error:", jnp.std(error))

In [ ]:
# Visualization of some examples
import matplotlib.pyplot as plt

k = 0

fig = plt.figure(figsize=(15, 4))
plt.subplot(1, 4, 1)
plt.title('Input')
plt.pcolor(u_downsammpled[k, :, :], cmap='jet')
plt.colorbar()

plt.subplot(1, 4, 2)
plt.title('Reference')
plt.pcolor(u_true[k, :, :], cmap='jet')
plt.colorbar()

plt.subplot(1, 4, 3)
plt.title('Prediction')
plt.pcolor(u_pred[k, :, :], cmap='jet')
plt.colorbar()

plt.subplot(1, 4, 4)
plt.title('Absolute Error')
plt.pcolor(jnp.abs(u_pred[k, :, :] - u_true[k, :, :]), cmap='jet')
plt.colorbar()

plt.tight_layout()
plt.show()